# Simulating the Mixed meal Model


In [8]:
using DifferentialEquations, SciMLBase, Plots, Random
using QuasiMonteCarlo
using Trapz
using MAT

# set random seed for reproducibility
Random.seed!(1234)


TaskLocalRNG()

In [9]:
include("Including.jl")

PlotMealResponse (generic function with 7 methods)

In [10]:
individual_1 = Individual(2)

# extract relevant data
BW_1 = individual_1.BW
fasting_glucose = individual_1.glc[1]
fasting_insulin = individual_1.ins[1]
G_dose = individual_1.G_dose



75000.0

In [11]:

model = EDES(fasting_glucose, fasting_insulin, G_dose, BW_1, estimated_params = ["k1","k5","k6", "k8"], timespan = (0.0, 480))

outputs = output(model)

(time = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0  …  471.0, 472.0, 473.0, 474.0, 475.0, 476.0, 477.0, 478.0, 479.0, 480.0], glucose_gut_to_plasma_flux = [0.0, 0.009299984439842923, 0.021987169270773807, 0.03491735794946587, 0.04724346383356455, 0.05866328600617034, 0.069095357111148, 0.07855787033556277, 0.08711333028663183, 0.09484166610735914  …  5.748462904264092e-5, 5.597861848520299e-5, 5.450682483186506e-5, 5.3054079949794375e-5, 5.163139169304613e-5, 5.024712558185602e-5, 4.8902549730933714e-5, 4.759680277824316e-5, 4.632689388500258e-5, 4.508770273568437e-5], hepatic_glucose_flux = [0.043, 0.04297554311908006, 0.04288019238367664, 0.04270538896921446, 0.042453289031300974, 0.04213009062145919, 0.041743744706428575, 0.04130287604117559, 0.04081621348417242, 0.040292284847489784  …  0.04404596127289292, 0.04404057240764279, 0.044035208311045466, 0.044029868760030136, 0.044024553840928564, 0.04401926362500308, 0.04401399809692517, 0.044008757210435, 0.04400354088834145, 0

In [12]:
model.prob.p

17-element Vector{Float64}:
     0.0105
     0.28
     0.00607
     0.000235
     0.0424
     2.2975
     0.4
     5.0
     0.06
     0.284
     1.4
    13.2
     4.44438211792684
    12.5404266739517
     0.043
 75000.0
    76.0

In [13]:
outputs.glucose_gut_to_plasma_flux
trapz(outputs.glucose_gut_to_plasma_flux, outputs.time)

f_G = 0.005551 # conversion factor for glucose, from mg/l to mmol/l
V_G = (260/sqrt(BW_1/70))/1000 # volume of distribution of glucose
answer = ((V_G*BW_1)/f_G)*trapz(outputs.time, outputs.glucose_gut_to_plasma_flux)*0.001

#plot(outputs.time, outputs.glucose_gut_to_plasma_flux, label = "glucose_gut_to_plasma_flux")

74.99326242070131

In [14]:
# four plots in a 2x2 grid showing plasma glucose, plasma insulin, plasma TG, and plasma NEFA
plot(outputs.time, [outputs.plasma_glucose, outputs.plasma_insulin], layout = (1, 2), label = ["glucose" "insulin"], xlabel = "time (min)", ylabel = ["glucose (mmol/L)" "insulin (uIU/ml)"])
savefig("glucose_insulin.png")

"c:\\Users\\20192809\\Documents\\GitHub\\EDES\\EDES\\glucose_insulin.png"

# Parameter estimation with the mixed meal model

In [15]:
using Optimization, OptimizationOptimJL, LineSearches

In [16]:
# get data
data = Individual(1);
# extract relevant data
BW_1 = individual_1.BW
fasting_glucose = individual_1.glc[1]
fasting_insulin = individual_1.ins[1]
G_dose = individual_1.G_dose
println(fasting_insulin)

model = EDES(fasting_glucose, fasting_insulin, estimated_params = ["k1","k5","k6","k8"], timespan = (0.0, 480))

12.5404266739517


EDES(ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, Vector{Float64}, ODEFunction{true, SciMLBase.AutoSpecialize, var"#29#30"{Float64, Float64, Float64, Float64, Float64, Float64}, LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, SymbolicIndexingInterface.SymbolCache{Nothing, Nothing, Nothing}, Nothing, Nothing}, Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}, SciMLBase.StandardODEProblem}(ODEFunction{true, SciMLBase.AutoSpecialize, var"#29#30"{Float64, Float64, Float64, Float64, Float64, Float64}, LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, SymbolicIndexingInterface.SymbolCache{Nothing, Nothing, Nothing}, Nothing, Nothing}(var"#29#30"{Float64, Float64, Float64, Float64, Float64, Float64}(0.1, 9.0, 3.0, 31.0

In [17]:
loss = make_LossFunction(model, data.glc, data.time, data.ins, data.time)

#select bounds for the latin hypercube sampler
lhc_ub = [0.5, 1.5, 10., 20.]
lhc_lb = [0.0005, 0, 0, 0.] 


# create preselected samples
initial_parameter_sets = Preselection_LHS(loss, model, 100, lhc_lb, lhc_ub)
initial_parameter_sets

4×10 Matrix{Float64}:
 0.0229775  0.0329675  0.257742  0.392607  …  0.0079925   0.362638   0.242757
 0.0825     0.0675     0.0225    0.7125       0.1425      0.1725     0.2475
 1.45       9.35       2.85      2.35         7.15        5.85       7.25
 2.1        4.5        9.5       1.9          5.3        10.9       11.7

First we need to make a loss function and get the best initial parameter sets using LHCS

Then we run the optimization for every parameter set:

In [18]:
optimizer = LBFGS(linesearch = BackTracking(order=3)) # cubic interpolation

objectives = []
parameters = []

opt_lb = lhc_lb
opt_ub = lhc_ub

for it in axes(initial_parameter_sets,2)
    try 
        optf = OptimizationFunction((x,p)-> loss(x), Optimization.AutoForwardDiff())
        starting_parameters = initial_parameter_sets[:,it]
        optprob = OptimizationProblem(optf, starting_parameters, lb = opt_lb, ub = opt_ub)
        sol = Optimization.solve(optprob, optimizer)

        push!(parameters, sol.u)
        push!(objectives, sol.objective)

    catch e
        throw(e)
        println("Optimization FAILED for this initial set, continuing...")
    end
end


In [19]:
objectives

10-element Vector{Any}:
  4.931815610404136
  4.931820344701956
 40.59334706442252
 95.25752448388533
  4.9318197206868994
 95.25757014696404
 95.28256963243346
  4.931814493495902
 40.59335488349495
 95.29230687755734

Get a full parameter vector:

In [20]:
full_parameter_vector = make_full_parameter_vector(model, parameters[2])

outputs = output(model, full_parameter_vector);
println(full_parameter_vector)


[0.021465830328348007, 0.28, 0.00607, 0.000235, 0.011808622099587884, 5.2414352240007185, 0.4, 5.318545987521016, 0.06, 0.284, 1.4, 13.2, 4.44438211792684, 12.5404266739517, 0.043, 75000.0, 70.0]


Finally let's plot the fitting

In [25]:
glucose_plot = plot(outputs.time, outputs.plasma_glucose, label = "glucose", xlabel = "time (min)", ylabel = "glucose (mmol/L)")
scatter!(data.time, data.glc, label = "glucose_data")

insulin_plot = plot(outputs.time, outputs.plasma_insulin, label = "insulin", xlabel = "time (min)", ylabel = "insulin (uIU/mL)")
scatter!(data.time, data.ins, label = "insulin_data")


plot(glucose_plot, insulin_plot, layout = (1, 2),legend=false)
savefig("glucose_insulin.png")

"c:\\Users\\20192809\\Documents\\GitHub\\EDES\\EDES\\glucose_insulin.png"